## PyO3

In [1]:
!pip uninstall felsenstein-rs -y
!maturin build --release
!pip install target/wheels/felsenstein_rs-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl

Found existing installation: felsenstein-rs 0.1.0
Uninstalling felsenstein-rs-0.1.0:
  Successfully uninstalled felsenstein-rs-0.1.0
🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /home/conda/miniconda3/envs/ML-cuda/bin/python3
📡 Using build options features from pyproject.toml
   Compiling autocfg v1.2.0
   Compiling proc-macro2 v1.0.81      ] 0/107: autocfg                     
   Compiling unicode-ident v1.0.12
   Compiling target-lexicon v0.12.14
   Compiling once_cell v1.19.0
   Compiling libc v0.2.155
   Compiling crossbeam-utils v0.8.19
   Compiling parking_lot_core v0.9.10
   Compiling paste v1.0.14
   Compiling hashbrown v0.14.3
   Compiling syn v1.0.109
   Compiling equivalent v1.0.1
   Compiling bytemuck v1.15.0
   Compiling rawpointer v0.2.1
   Compiling portable-atomic v1.6.0
   Compiling cfg-if v1.0.0
   Compiling heck v0.4.1
   Compiling typenum v1.17.0
   Compiling smallvec v1.13.2
   Compiling winnow v0.5.40
   Compiling toml_datetime v0.6.5     ] 0/107: unicode-ident, 

In [2]:
import numpy as np
from itertools import combinations
import felsenstein_rs as frs

In [3]:
ftree = frs.FTree("data/tree.txt", 1e-4)

In [4]:
#all_index_pairs = np.array([[*tup] for tup in combinations(range(119), 2)], dtype=np.uint64)

In [5]:
# TODO accept dtype=int instead of uint
index_pairs = np.array([[0, 1], [0, 2], [0, 3]], dtype=np.uint64)

In [6]:
rate_matrix = -np.eye(16) + (np.ones((16, 16)) - np.eye(16)) * (1/15)
rate_matrices = np.broadcast_to(rate_matrix, (index_pairs.shape[0],) + rate_matrix.shape)

In [7]:
log_p_prior = np.log(np.ones((16,)) * (1/16))
log_p_priors = np.broadcast_to(log_p_prior, (index_pairs.shape[0],) + log_p_prior.shape)

In [8]:
#ftree.debug()

In [9]:
result = ftree.infer(index_pairs, rate_matrices, log_p_priors)

In [10]:
result[1][0]

array([[  0.36302841,   0.85550747,   1.17086794,   0.67957863,
          0.27386764,   1.09975427,   0.39673205,   0.83557911,
          0.63480082,   1.34024767,   1.3794857 ,   1.26391303,
          1.90526616,   1.00928327,   1.82906964,   1.02972399],
       [  0.54426   ,   4.82765326,   1.30920805,  15.57764687,
          0.45508891,   1.28161728,   0.58510799,   1.01690178,
          0.81633207,   5.76908687,   1.5839582 ,   1.43990081,
          0.652363  ,   1.13381507,   2.00478677,   1.21753424],
       [ 15.45404892,  29.54135651,   5.21717177,  44.52523992,
          1.3113909 ,   2.15851647,   2.80158745,   2.52825672,
         18.73079595,   8.55181157,  16.58973111,  16.64455016,
          9.27551223,  16.10246537,   3.05283921,  15.22110903],
       [  0.74702252,  13.7621568 ,  15.78587671,   4.45185919,
          5.06187742,   1.47920189,   0.77640774,   1.22234656,
          1.01425798,   1.78835249,   1.76551344,   1.64610554,
          0.96139522,   1.34056993,  

In [12]:
# This now gives the correct error message
#ftree_bad = frs.FTree("data/tree_topological.csv", 1e-4)